In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import collections
import time
import warnings
from sys import getsizeof
warnings.filterwarnings('ignore')


In [3]:
df = pd.read_csv("/workspaces/codespaces-jupyter/data/data_for_recommendations.csv")
df.shape

(14969, 28)

In [4]:
df.to_csv("data_for_recommendations.csv",index = False)

In [5]:
df_diet = df[['title','ingredients','diet_type','URL','description']]

In [6]:
df_diet.head(10)

,title,ingredients,diet_type,URL,description
0,Korean-Style Seaweed Soup,"['sauce', 'seaweed', 'water', 'sirloin', 'salt...",nveg,https://www.allrecipes.com/recipe/66294/korean...,Seaweed soup made with brown seaweed and beef ...
1,Korean Tofu and Vegetable Soup,"['onions', 'x', 'kombu', 'squash', 'tofu', 'pe...",nveg,https://www.allrecipes.com/recipe/55528/korean...,This is the Korean equivalent of chicken noodl...
2,Korean Soft Tofu Stew (Soon Du Bu Jigae),"['onion', 'bean', 'seeds', 'tofu', 'pepper', '...",nveg,https://www.allrecipes.com/recipe/151006/korea...,This versatile tofu stew is hearty and simple ...
3,Quick and Simple Korean Doenjang Chigae (Bean ...,"['zucchini', 'bean', 'onions', 'tofu', 'pepper...",vegan,https://www.allrecipes.com/recipe/212699/quick...,This Korean soybean soup with tofu is a simple...
4,Korean Beef Stew,"['pepper', 'ground', 'butter', 'sesame', 'soy'...",nveg,https://www.allrecipes.com/recipe/273203/korea...,This one-pan recipe creates a Korean beef stew...
5,Korean Miso Soup,"['onion', 'zucchini', 'gochujang', 'bean', 'to...",veg,https://www.allrecipes.com/recipe/25753/korean...,This tasty Korean miso soup made with bean cur...
6,Budae Jjigae (Korean Army Base Stew),"['onion', 'spam', 'pepper', 'ground', 'ramen',...",nveg,https://www.allrecipes.com/recipe/255397/budae...,This spicy budae jjigae stew is a traditional ...
7,Korean Beef Short Rib Stew (Galbi Jjim),"['onion', 'corn', 'syrup', 'chuck', 'dates', '...",nveg,https://www.allrecipes.com/recipe/185788/korea...,"A classic Korean comfort food, these braised b..."
8,Dakdoritang (Korean Spicy Chicken Stew),"['onion', 'pepper', 'ground', 'gochugaru', 'fa...",nveg,https://www.allrecipes.com/recipe/236239/dakdo...,'Dakdoritang' is a Korean chicken dish that's ...
9,Kimchi Jigae (Kimchee Soup),"['pork', 'spam', 'tofu', 'water', 'mushrooms',...",nveg,https://www.allrecipes.com/recipe/221554/kimch...,Add some warmth to your dinner table on winter...


In [7]:
df_diet['title_encoded'] = df_diet.index

In [8]:
title_codes = {}
n = 0
for i in list(df_diet.title):
    title_codes[i] = n
    n+=1
    

In [9]:
def convert_tolist(x):
    y = eval(x)
    return y

In [10]:
df_diet.ingredients = df_diet.ingredients.apply(convert_tolist)

In [11]:
def list_to_string(ingredient_list):
    return ' '.join(ingredient_list)

In [12]:
df_diet.ingredients = df_diet.ingredients.apply(list_to_string)

In [13]:
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.similarities import MatrixSimilarity


In [14]:
df_diet_non_veg = df_diet[df_diet['diet_type'] == 'nveg'].reset_index(drop = True)
df_diet_veg = df_diet[df_diet['diet_type'] == 'veg'].reset_index(drop = True)
df_diet_vegan = df_diet[df_diet['diet_type'] == 'vegan'].reset_index(drop = True)


# SUPRISE-ME

In [15]:

# Create a dictionary from the one-hot encoded ingredients
ingredient_dict = Dictionary(df_diet['ingredients'].apply(lambda x: x.split()))

# Create a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict.doc2bow(recipe.split()) for recipe in df_diet['ingredients']]

# Create an LSI topic model from the ingredient data
model_0 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict, num_topics=50)

# Create a matrix similarity index for the topic model
index_0 = MatrixSimilarity(model_0[ingredient_bow])



# NON-VEG

In [16]:

# Create a dictionary from the one-hot encoded ingredients
ingredient_dict = Dictionary(df_diet_non_veg['ingredients'].apply(lambda x: x.split()))

# Create a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict.doc2bow(recipe.split()) for recipe in df_diet_non_veg['ingredients']]

# Create an LSI topic model from the ingredient data
model_1 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict, num_topics=50)

# Create a matrix similarity index for the topic model
index_1 = MatrixSimilarity(model_1[ingredient_bow])



# VEG

In [17]:

# Create a dictionary from the one-hot encoded ingredients
ingredient_dict = Dictionary(df_diet_veg['ingredients'].apply(lambda x: x.split()))

# Create a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict.doc2bow(recipe.split()) for recipe in df_diet_veg['ingredients']]

# Create an LSI topic model from the ingredient data
model_2 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict, num_topics=50)

# Create a matrix similarity index for the topic model
index_2 = MatrixSimilarity(model_2[ingredient_bow])



# VEGAN

In [18]:

# Create a dictionary from the one-hot encoded ingredients
ingredient_dict = Dictionary(df_diet_vegan['ingredients'].apply(lambda x: x.split()))

# Create a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict.doc2bow(recipe.split()) for recipe in df_diet_vegan['ingredients']]

# Create an LSI topic model from the ingredient data
model_3 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict, num_topics=50)

# Create a matrix similarity index for the topic model
index_3 = MatrixSimilarity(model_3[ingredient_bow])


In [19]:
model_dict = {"suprise":0,"non-veg":1,"veg":2,"vegan":3}

In [20]:
# Define a function to get the top n similar recipes based on user ingredients
def get_recipes_by_ingredients(ingredients, n,model):
    if model_dict.get(model) == 0:
        # Convert the user ingredients to bag-of-words representation
        ingredient_bow = ingredient_dict.doc2bow(ingredients)

        # Get the topic distribution for the user ingredients
        user_topics = model_0[ingredient_bow]

        # Get the similarity scores for all recipes based on user ingredients
        sim_scores = index_0[user_topics]

        # Sort the recipes by similarity score
        sim_scores = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)

        # Get the top n similar recipes
        sim_scores = sim_scores[0:n]
        recipe_indices = [i[0] for i in sim_scores]
        return df_diet.iloc[recipe_indices]
    if model_dict.get(model) == 1:
        # Convert the user ingredients to bag-of-words representation
        ingredient_bow = ingredient_dict.doc2bow(ingredients)

        # Get the topic distribution for the user ingredients
        user_topics = model_1[ingredient_bow]

        # Get the similarity scores for all recipes based on user ingredients
        sim_scores = index_1[user_topics]

        # Sort the recipes by similarity score
        sim_scores = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)

        # Get the top n similar recipes
        sim_scores = sim_scores[0:n]
        recipe_indices = [i[0] for i in sim_scores]
        return df_diet_non_veg.iloc[recipe_indices]
    if model_dict.get(model) == 2:
        # Convert the user ingredients to bag-of-words representation
        ingredient_bow = ingredient_dict.doc2bow(ingredients)

        # Get the topic distribution for the user ingredients
        user_topics = model_2[ingredient_bow]

        # Get the similarity scores for all recipes based on user ingredients
        sim_scores = index_2[user_topics]

        # Sort the recipes by similarity score
        sim_scores = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)

        # Get the top n similar recipes
        sim_scores = sim_scores[0:n]
        recipe_indices = [i[0] for i in sim_scores]
        return df_diet_veg.iloc[recipe_indices]
    if model_dict.get(model) == 3:
        # Convert the user ingredients to bag-of-words representation
        ingredient_bow = ingredient_dict.doc2bow(ingredients)

        # Get the topic distribution for the user ingredients
        user_topics = model_3[ingredient_bow]

        # Get the similarity scores for all recipes based on user ingredients
        sim_scores = index_3[user_topics]

        # Sort the recipes by similarity score
        sim_scores = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)

        # Get the top n similar recipes
        sim_scores = sim_scores[0:n]
        recipe_indices = [i[0] for i in sim_scores]
        return df_diet_vegan.iloc[recipe_indices]

# Test the function with a list of user ingredients and get the top 3 similar recipes



In [21]:
ingredients = ['chicken', 'potato','cheese']
get_recipes_by_ingredients(ingredients, n=10,model ='vegan')

,title,ingredients,diet_type,URL,description,title_encoded
1743,Cassava Couscous (Attieke),onion cilantro seeds g peanuts water sesame le...,vegan,https://www.allrecipes.com/recipe/280918/cassa...,"Fermented cassava couscous, or attieke, is a t...",8566
2285,Mandarin Pumpkins (Healthy Halloween Snack),mandarin pith stalks clementine celery oranges,vegan,https://www.allrecipes.com/recipe/260519/manda...,If you are looking for a healthy Halloween sna...,12977
1950,Carrot and Apple Juice,apples carrots stalks celery ginger,vegan,https://www.allrecipes.com/recipe/244513/carro...,Fresh carrot and apple juice with a hint of gi...,10605
1038,Vegetarian Sheet Pan Dinner with Chickpeas and...,potatoes onion seeds onions potato pepper russ...,vegan,https://www.allrecipes.com/recipe/268995/veget...,Adding chickpeas to root vegetables on a sheet...,3913
1419,Cauliflower and Potato Stir-Fry - East Indian ...,onion seeds cilantro potato turmeric cauliflow...,vegan,https://www.allrecipes.com/recipe/222670/cauli...,Cauliflower and potatoes are simmered with aro...,6844
2189,Hawaij Vegetable Soup,onion tomato beans potato pepper water hawaij ...,vegan,https://www.allrecipes.com/recipe/213660/hawai...,"Hawaij, a Yemenite curry blend, makes a delici...",12297
589,Watermelon Refreshing Green Smoothie,watermelon apple broccoli cucumber celery flor...,vegan,https://www.allrecipes.com/recipe/260168/water...,Start your day with this watermelon spinach sm...,2404
1070,Chinese Chicken Salad,halves lettuce briefly onions seeds breast bon...,vegan,https://www.allrecipes.com/recipe/8914/chinese...,A wonderful Chinese-inspired chicken salad rec...,4255
1139,Best Broccoli Salad,apples seeds broccoli cauliflower catalina sun...,vegan,https://www.allrecipes.com/recipe/16483/best-b...,Florets of broccoli and cauliflower are mixed ...,4707
2390,Minestrone Soup,kernels onion zucchini corn beans tomatoes pot...,vegan,https://www.allrecipes.com/recipe/14118/minest...,This soup uses vegetable stock for the base an...,13765


In [1]:
ingredients = ['apple','celery']
get_recipes_by_ingredients(ingredients, n=10,model ='vegan')

NameError: name 'get_recipes_by_ingredients' is not defined

In [27]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 66.5 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 35.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 65.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.1/340.1 kB 15.2 MB/s eta 0:00:00
  Created wheel for validators: filename=val

In [2]:
pip install folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import folium

m = folium.Map()
m.save("footprint.html")

In [4]:
m